In [1]:
import libsbml
import os

from libsbml import CVTerm

In [2]:
def convert(file):
    doc = libsbml.readSBMLFromFile(file)
    model = doc.getModel()

    # enable fbc plugin
    if not doc.isPackageEnabled("fbc"):
        doc.enablePackage("http://www.sbml.org/sbml/level3/version1/fbc/version2", "fbc", True)
    fbc_plugin = model.getPlugin("fbc")
    fbc_plugin.setStrict(False)

    # get charge and formula from nodes, add as fbc-attribute
    for species in model.getListOfSpecies():
        fbc_species_plugin = species.getPlugin("fbc")
        notes = species.getNotesString()
        splits = notes.split("<p>")
        formula = splits[1][9:].strip()[:-4].strip()
        charge = splits[2][8:].split("</p>")[0].strip()
        if formula:
            fbc_species_plugin.setChemicalFormula(formula)
        if charge and charge != 'NaN':
            fbc_species_plugin.setCharge(int(charge))
        
    for reaction in model.getListOfReactions():
        fbc_reaction_plugin = reaction.getPlugin("fbc")
        reaction.setMetaId(reaction.id)
        # get subsystem and EC number
        notes = reaction.getNotesString()
        splits = notes.split("<p>")
        ec_numbers = splits[3][11:].strip()[:-4]
        
        # biological qualifier
        cv_term = CVTerm(1)
        # BQB_HAS_PROPERTY
        cv_term.setBiologicalQualifierType(10)
        # Add enzyme comission numbers as MIRIAM identifiers
        if ec_numbers:
            if "," in ec_numbers:
                for ec_num in ec_numbers.split(","):
                    cv_term.addResource(f"http://identifiers.org/ec-code/{ec_num}")
            else:
                cv_term.addResource(f"http://identifiers.org/ec-code/{ec_numbers}")
        if cv_term.getNumResources() > 0:
            reaction.addCVTerm(cv_term)
    
        # transfer flux bound information to fbc
        kinetic_law = reaction.getKineticLaw()
    
        # convert local bound paramters to global parameters
        for bound_type in ["LOWER_BOUND", "UPPER_BOUND"]:
            bound = kinetic_law.getLocalParameter(bound_type)
            param = libsbml.Parameter(3,1)
            param.setConstant(True)
            r_id = reaction.id if not reaction.id.startswith("R_") else reaction.id[2:] 
            param.setId("{}_{}".format(bound_type, r_id))
            param.setName(param.getId())
            param.setValue(bound.getValue())
            model.addParameter(param)
            if bound_type == "LOWER_BOUND":
                fbc_reaction_plugin.setLowerFluxBound(param.id)
            else:
                fbc_reaction_plugin.setUpperFluxBound(param.id)
    
    out_file = os.path.basename(file).split(".")[0] + "_fbc.xml"
    libsbml.writeSBMLToFile(doc, os.path.join("fbc", out_file))

In [3]:
files = ["iPAE1146_Amino_sugar_and_nucleotide_sugar_metabolism.xml",
         "iPAE1146_Lipopolysaccharide_biosynthesis.xml",
         "iPAE1146_Pyrimidine_metabolism.xml",
         "iPAE1146_Combined_Subsystems.xml"]
for file in files:
    convert(os.path.join("xml", file))